In [1]:
import mysql.connector
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import unicodedata
import re
from datetime import timedelta, datetime
from rapidfuzz import fuzz
from sklearn.impute import KNNImputer
from typing import Dict
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
pd.set_option("display.max_columns", None)

Run etl.ipynb

In [3]:
load_dotenv()

# Event data
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    database=os.getenv("DB_NAME"),
    password=os.getenv("DB_PASSWORD")
    )

cursor = conn.cursor()

query = ("SELECT * FROM events")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_events = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

In [4]:
# Convert to datetime
df_events['event_date'] = pd.to_datetime(df_events['event_date'], format="%Y-%m-%d")
# Drop duplicates
df_events[df_events.duplicated(keep=False)]
df_events = df_events.drop_duplicates()
# Drop where fights are null
df_events = df_events[~((df_events['fighter_red'].isna()) & (df_events['fighter_blue'].isna()))]
# Drop where winner is null
df_events = df_events[~df_events[['winner']].isnull().all(axis=1)]


# Fill stances
df_fighters_red = df_events[['fighter_red', 'stance_red']].rename(
    columns={'fighter_red': 'fighter', 'stance_red': 'stance'}
)
df_fighters_blue = df_events[['fighter_blue', 'stance_blue']].rename(
    columns={'fighter_blue': 'fighter', 'stance_blue': 'stance'}
)

# Combine into one fighter dataframe
df_fighters = pd.concat([df_fighters_red, df_fighters_blue], ignore_index=True)

# Drop duplicate fighters (keep their first known stance)
df_fighters = df_fighters.drop_duplicates(subset=['fighter'], keep='first')

# Count unique stances across fighters
stance_counts = df_fighters['stance'].value_counts(dropna=True)

top_stances = stance_counts.index[:1].tolist()

df_events['stance_red'] = df_events['stance_red'].fillna(top_stances[0])
df_events['stance_blue'] = df_events['stance_blue'].fillna(top_stances[0])


# Convert to int
for col in df_events.select_dtypes(include=['number']).columns:
    df_events[col] = df_events[col].astype('int64')
    
# Change winner column to bolean
df_events['winner'] = (df_events['winner'] == df_events['fighter_red']).astype(int)

# Normalize weight classes
def clean_text(val: str) -> str:
    if not isinstance(val, str):
        return val
    val = val.strip().lower()  # lowercase + trim
    val = re.sub(r"\s+", " ", val)  # collapse multiple spaces
    val = val.replace("womens", "women's")  # normalize missing apostrophe
    val = val.replace("women ", "women's ") # normalize if missing "'s"
    return val

# master map of cleaned values → canonical
mapping = {
    "lightweight": "Lightweight",
    "welterweight": "Welterweight",
    "middleweight": "Middleweight",
    "featherweight": "Featherweight",
    "bantamweight": "Bantamweight",
    "heavyweight": "Heavyweight",
    "light heavyweight": "Light Heavyweight",
    "flyweight": "Flyweight",
    "women's strawweight": "Women's Strawweight",
    "women's flyweight": "Women's Flyweight",
    "women's bantamweight": "Women's Bantamweight",
    "open weight": "Open Weight",
    "catch weight": "Catch Weight"
}

def normalize_weight_class(val):
    # clean first
    cleaned = clean_text(val)

    # if it matches exactly after cleaning
    if cleaned in mapping:
        return mapping[cleaned]
    
    # fuzzy keyword matching for tournament-style
    if "heavyweight" in cleaned and "light" not in cleaned:
        return "Heavyweight"
    elif "lightweight" in cleaned and "feather" not in cleaned:
        return "Lightweight"
    elif "middleweight" in cleaned:
        return "Middleweight"
    elif "featherweight" in cleaned:
        return "Featherweight"
    elif "bantamweight" in cleaned:
        # if it’s a women’s version
        if "women" in cleaned:
            return "Women's Bantamweight"
        return "Bantamweight"
    elif "flyweight" in cleaned:
        if "women" in cleaned:
            return "Women's Flyweight"
        return "Flyweight"
    elif "strawweight" in cleaned:
        return "Women's Strawweight"
    
    # default
    return "Open Weight"

df_events["weight_class"] = df_events["weight_class"].apply(normalize_weight_class)

df_events.drop(columns=['id'], axis=1, inplace=True)

df_events = df_events.reset_index(drop=True)

# Normalize names
def normalize_name(name):

    if pd.isna(name):
        return ""
    
    # Convert to string if not already
    name = str(name)

    # Remove extra whitespace
    #name = name.strip()

    # Normalize unicode characters
    name = unicodedata.normalize('NFKD', name)
    name = ''.join(c for c in name if not unicodedata.combining(c))

    # Convert to lower
    name = name.lower()

    # Remove apostrophes and replace with nothing or space
    #name = re.sub(r"'", "", name)

    # Remove periods and other punctuation
    name = re.sub(r'[^\w\s-]', '', name)

    # Normalize spaces (multiple spaces to single space)
    name = re.sub(r'\s+', ' ', name)

    words = name.split()

    return ' '.join(words).strip()

df_events[["fighter_red", "fighter_blue"]] = df_events[["fighter_red", "fighter_blue"]].map(normalize_name)

events_cleaned = df_events.copy()

In [5]:
# Stats data
conn = mysql.connector.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    database=os.getenv("DB_NAME"),
    password=os.getenv("DB_PASSWORD")
    )

cursor = conn.cursor()

query = ("SELECT * FROM stats")

cursor.execute(query)

columns = [desc[0] for desc in cursor.description]

rows = cursor.fetchall()

df_stats = pd.DataFrame(rows, columns=columns)

cursor.close()
conn.close()

In [6]:
# Drop duplicate names
df_stats = df_stats.assign(nan_count=df_stats.isnull().sum(axis=1)) \
            .sort_values(['name', 'nan_count']) \
            .drop_duplicates('name', keep='first') \
            .drop('nan_count', axis=1)

# Keep necessary columns
df_stats = df_stats[['name', 'octagon_debut', 'height', 'weight', 'reach',
    'leg_reach', 'sig_strikes_landed_per_minute', 'sig_strikes_absorbed_per_minute', 
    'takedowns_avg', 'submission_avg', 'knockdown_avg', 'fight_time_avg']]

# Fill with null the 0 values
df_stats[['height', 'weight']] = df_stats[['height', 'weight']].replace(0, np.nan)

# Fill with 0
df_stats[['sig_strikes_landed_per_minute', 'sig_strikes_absorbed_per_minute','takedowns_avg',
        'submission_avg','knockdown_avg']] = df_stats[['sig_strikes_landed_per_minute', 
                                        'sig_strikes_absorbed_per_minute','takedowns_avg','submission_avg',
                                        'knockdown_avg']].fillna(0)

# Fill null
df_stats[['fight_time_avg']] = df_stats[['fight_time_avg']].fillna('00:00')

# Convert time to seconds
df_stats['fight_time_avg'] = pd.to_timedelta('00:' + df_stats['fight_time_avg']).dt.total_seconds().astype(int)

# Impute missing values
# Load the fitted imputer
with open("../models/knn_imputer_stats.pkl", "rb") as f:
    imputer = pickle.load(f)
missing_cols = df_stats.columns[df_stats.isnull().any()]

# Impute using the existing fitted imputer
df_stats[missing_cols] = imputer.transform(df_stats[missing_cols])

# Normalize names
df_stats["name"] = df_stats["name"].map(normalize_name)

stats_cleaned = df_stats.copy()

Feature_engineering.ipynb

In [7]:
# Fuzzy match
def create_fuzzy_mapping(event_names, stats_names, threshold=85):
    """
    Create a mapping dictionary from events fighter names to stats names
    using fuzzy matching.
    """
    mapping = {}

    for event_name in event_names:
        if pd.isna(event_name):
            continue

        best_match = None
        best_score = 0

        for stats_name in stats_names:
            if pd.isna(stats_name):
                continue

            # Calculate fuzzy match scores
            ratio = fuzz.ratio(event_name, stats_name)
            token_sort_ratio = fuzz.token_sort_ratio(event_name, stats_name)
            token_set_ratio = fuzz.token_set_ratio(event_name, stats_name)
            score = max(ratio, token_sort_ratio, token_set_ratio)
            
            if score >= threshold and score > best_score:
                best_match = stats_name
                best_score = score

        if best_match:
            mapping[event_name] = best_match
        else:
            mapping[event_name] = event_name # Keep original if no match found

    return mapping

# Get unique fighter names from events
blue_fighters = events_cleaned['fighter_blue'].dropna().unique()
red_fighters = events_cleaned['fighter_red'].dropna().unique()
all_event_fighters = set(blue_fighters) | set(red_fighters)

# Get unique names from stats
stats_names = stats_cleaned['name'].dropna().unique()

# Create fuzzy mapping
print("Creating fuzzy mapping...")
fuzzy_mapping = create_fuzzy_mapping(all_event_fighters, stats_names, threshold=85)

# Apply fuzzy mapping to create new columns for merging
events_cleaned['fighter_blue_mapped'] = events_cleaned['fighter_blue'].map(fuzzy_mapping)
events_cleaned['fighter_red_mapped'] = events_cleaned['fighter_red'].map(fuzzy_mapping)

# Fill NaN values with original names (for cases where mapping didn't work)
events_cleaned['fighter_blue_mapped'] = events_cleaned['fighter_blue_mapped'].fillna(events_cleaned['fighter_blue'])
events_cleaned['fighter_red_mapped'] = events_cleaned['fighter_red_mapped'].fillna(events_cleaned['fighter_red'])

# Now perform the merges using the mapped names
# Merge stats for fighter_blue
df_merged = events_cleaned.merge(
    stats_cleaned,
    how='left',
    left_on='fighter_blue_mapped',
    right_on='name',
    suffixes=('', '_drop')
)

# Rename fighter_blue stats columns with _blue suffix (excluding the original 'name')
cols_to_rename_blue = [col for col in stats_cleaned.columns if col != 'name']
df_merged.rename(columns={col: f"{col}_blue" for col in cols_to_rename_blue}, inplace=True)
df_merged.drop(columns=['name', 'fighter_blue_mapped'], inplace=True)

# Merge stats for fighter_red
df_merged = df_merged.merge(
    stats_cleaned,
    how='left',
    left_on='fighter_red_mapped',
    right_on='name',
    suffixes=('', '_drop')
)

# Rename fighter_red stats columns with _red suffix
cols_to_rename_red = [col for col in stats_cleaned.columns if col != 'name']
df_merged.rename(columns={col: f"{col}_red" for col in cols_to_rename_red}, inplace=True)
df_merged.drop(columns=['name', 'fighter_red_mapped'], inplace=True)

# Fill octagon_debut with min value of the event_date of the fighter
df_merged['octagon_debut_blue'] = pd.to_datetime(df_merged['octagon_debut_blue'], errors='coerce')
df_merged['octagon_debut_red'] = pd.to_datetime(df_merged['octagon_debut_red'], errors='coerce')

fighters_long = pd.concat([
    events_cleaned[['fighter_red', 'event_date']].rename(columns={'fighter_red': 'fighter'}),
    events_cleaned[['fighter_blue', 'event_date']].rename(columns={'fighter_blue': 'fighter'})
], ignore_index=True)

fighter_debuts = fighters_long.groupby('fighter')['event_date'].min().reset_index()
fighter_debuts.rename(columns={'event_date': 'octagon_debut'}, inplace=True)

df_merged = df_merged.merge(
    fighter_debuts, left_on='fighter_red', right_on='fighter', how='left'
)

df_merged.loc[df_merged['octagon_debut_red'].isna(), 'octagon_debut_red'] = \
    df_merged.loc[df_merged['octagon_debut_red'].isna(), 'octagon_debut']

df_merged.drop(columns='fighter', inplace=True)
df_merged.drop(columns='octagon_debut', inplace=True)

df_merged = df_merged.merge(
    fighter_debuts, left_on='fighter_blue', right_on='fighter', how='left'
)

df_merged.loc[df_merged['octagon_debut_blue'].isna(), 'octagon_debut_blue'] = \
    df_merged.loc[df_merged['octagon_debut_blue'].isna(), 'octagon_debut']

df_merged.drop(columns='fighter', inplace=True)
df_merged.drop(columns='octagon_debut', inplace=True)

# Impute missing values
# Load the fitted imputer
with open("../models/knn_imputer_feature_engineering.pkl", "rb") as f:
    imputer = pickle.load(f)
missing_cols = df_merged.columns[df_merged.isnull().any()]

df_merged[missing_cols] = imputer.fit_transform(df_merged[missing_cols])

# Convert to datetime
df_merged['event_date'] = pd.to_datetime(df_merged['event_date'], errors='coerce')

df_merged.sort_values(by=['event_date'], inplace=True)
df_merged.reset_index(drop=True, inplace=True)

Creating fuzzy mapping...


Gets name to predict and add some features

In [8]:
names = ['kelvin gastelum', 'dustin stoltzfus']

df_filtered = df_merged[
    (df_merged['fighter_red'].isin(names)) |
    (df_merged['fighter_blue'].isin(names))
]

In [9]:
# Define columns for each fighter
red_columns = [
    'sig_strikes_landed_per_minute_red', 'sig_strikes_absorbed_per_minute_red', 
    'takedowns_avg_red', 'submission_avg_red', 'knockdown_avg_red', 'fight_time_avg_red',
    'stance_red', 'octagon_debut_red', 'height_red', 'weight_red', 'reach_red', 'leg_reach_red'
]

blue_columns = [
    'sig_strikes_landed_per_minute_blue', 'sig_strikes_absorbed_per_minute_blue', 
    'takedowns_avg_blue', 'submission_avg_blue', 'knockdown_avg_blue', 'fight_time_avg_blue',
    'stance_blue', 'octagon_debut_blue', 'height_blue', 'weight_blue', 'reach_blue', 'leg_reach_blue'
]

# Get the last row where diego lopes was fighter_red
last_red_row = df_merged[df_merged['fighter_red'] == names[0]].iloc[-1] if len(df_merged[df_merged['fighter_red'] == names[0]]) > 0 else None

# Get the last row where jean silva was fighter_blue  
last_blue_row = df_merged[df_merged['fighter_blue'] == names[1]].iloc[-1] if len(df_merged[df_merged['fighter_blue'] == names[1]]) > 0 else None

# Create new row data
new_row_data = {
    'event_date': pd.to_datetime('today').date(),
    'fighter_red': names[0],  # 'diego lopes'
    'fighter_blue': names[1], # 'jean silva'
}

# Copy red fighter stats from their last fight as red fighter
if last_red_row is not None:
    for col in red_columns:
        if col in df_merged.columns:
            new_row_data[col] = last_red_row[col]

# Copy blue fighter stats from their last fight as blue fighter
if last_blue_row is not None:
    for col in blue_columns:
        if col in df_merged.columns:
            new_row_data[col] = last_blue_row[col]
    # Also copy weight_class from blue fighter's last fight
    if 'weight_class' in df_merged.columns:
        new_row_data['weight_class'] = last_blue_row['weight_class']

# Add the new row
df_filtered = pd.concat([df_filtered, pd.DataFrame([new_row_data])], ignore_index=True)

# Ensure event_date is datetime
df_filtered['event_date'] = pd.to_datetime(df_filtered['event_date'], errors='coerce').dt.date

df_filtered.tail(1)

,event_date,event_name,fighter_red,fighter_blue,round,time,weight_class,win_method,winner,stance_red,stance_blue,knockdowns_red,knockdowns_blue,sig_attempts_red,sig_attempts_blue,sig_strikes_red,sig_strikes_blue,total_strikes_attempts_red,total_strikes_attempts_blue,total_strikes_red,total_strikes_blue,sub_attempts_red,sub_attempts_blue,takedowns_red,takedowns_blue,takedown_attempts_red,takedown_attempts_blue,control_time_red,control_time_blue,head_strikes_red,head_strikes_blue,head_attempts_red,head_attempts_blue,body_strikes_red,body_strikes_blue,body_attempts_red,body_attempts_blue,leg_strikes_red,leg_strikes_blue,leg_attempts_red,leg_attempts_blue,distance_red,distance_blue,distance_attempts_red,distance_attempts_blue,clinch_strikes_red,clinch_strikes_blue,clinch_attempts_red,clinch_attempts_blue,ground_strikes_red,ground_strikes_blue,ground_attempts_red,ground_attempts_blue,octagon_debut_blue,height_blue,weight_blue,reach_blue,leg_reach_blue,sig_strikes_landed_per_minute_blue,sig_strikes_absorbed_per_minute_blue,takedowns_avg_blue,submission_avg_blue,knockdown_avg_blue,fight_time_avg_blue,octagon_debut_red,height_red,weight_red,reach_red,leg_reach_red,sig_strikes_landed_per_minute_red,sig_strikes_absorbed_per_minute_red,takedowns_avg_red,submission_avg_red,knockdown_avg_red,fight_time_avg_red
32,2025-09-21,NaN,kelvin gastelum,dustin stoltzfus,NaN,NaN,Middleweight,NaN,NaN,Southpaw,Orthodox,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-21,72.0,185.5,75.0,43.5,3.0,3.0,2.26,1.29,0.32,558.0,2013-04-14,69.0,185.0,71.5,39.5,4.0,3.0,1.05,0.09,0.41,820.0


Create average value per stat feature

In [10]:
red_fights = df_filtered[['event_date','event_name','fighter_red','round','time','weight_class','win_method','winner','stance_red',
                        'knockdowns_red','sig_attempts_red','sig_strikes_red','total_strikes_attempts_red','total_strikes_red',
                        'sub_attempts_red','takedowns_red','takedown_attempts_red','control_time_red','head_strikes_red','head_attempts_red',
                        'body_strikes_red','body_attempts_red','leg_strikes_red','leg_attempts_red','distance_red','distance_attempts_red',
                        'clinch_strikes_red','clinch_attempts_red','ground_strikes_red','ground_attempts_red',]].copy()

# Rename columns but keep fighter_red as fighter_name to avoid conflict
red_fights.columns = ['fighter_name' if col == 'fighter_red' else col.replace('_red', '') if '_red' in col else col for col in red_fights.columns]
red_fights.columns = [col.replace('_red', '') if '_red' in col else col for col in red_fights.columns]
red_fights['corner'] = 'red'

blue_fights = df_filtered[['event_date','event_name','fighter_blue','round','time','weight_class','win_method','winner','stance_blue',
                        'knockdowns_blue','sig_attempts_blue','sig_strikes_blue','total_strikes_attempts_blue','total_strikes_blue',
                        'sub_attempts_blue','takedowns_blue','takedown_attempts_blue','control_time_blue','head_strikes_blue','head_attempts_blue',
                        'body_strikes_blue','body_attempts_blue','leg_strikes_blue','leg_attempts_blue','distance_blue','distance_attempts_blue',
                        'clinch_strikes_blue','clinch_attempts_blue','ground_strikes_blue','ground_attempts_blue',]].copy()

blue_fights.columns = ['fighter_name' if col == 'fighter_blue' else col.replace('_blue', '') if '_blue' in col else col for col in blue_fights.columns]
blue_fights.columns = [col.replace('_blue', '') if '_blue' in col else col for col in blue_fights.columns]
blue_fights['corner'] = 'blue'

red_fights['opponent'] = df_filtered['fighter_blue']
blue_fights['opponent'] = df_filtered['fighter_red']

all_fights = pd.concat([red_fights, blue_fights]).sort_values(['fighter_name', 'event_date'])

def calculate_fighter_features(df):

    df = df.sort_values('event_date')

    # Calculate expanding/rolling features (using previous fights only)
    df['avg_knockdowns'] = df['knockdowns'].expanding().mean().shift(1)
    df['avg_sig_attempts'] = df['sig_attempts'].expanding().mean().shift(1)
    df['avg_sig_strikes'] = df['sig_strikes'].expanding().mean().shift(1)
    df['avg_total_strikes_attempts'] = df['total_strikes_attempts'].expanding().mean().shift(1)
    df['avg_total_strikes'] = df['total_strikes'].expanding().mean().shift(1)
    df['avg_sub_attempts'] = df['sub_attempts'].expanding().mean().shift(1)
    df['avg_takedowns'] = df['takedowns'].expanding().mean().shift(1)
    df['avg_takedown_attempts'] = df['takedown_attempts'].expanding().mean().shift(1)
    df['avg_head_strikes'] = df['head_strikes'].expanding().mean().shift(1)
    df['avg_head_attempts'] = df['head_attempts'].expanding().mean().shift(1)
    df['avg_body_strikes'] = df['body_strikes'].expanding().mean().shift(1)
    df['avg_body_attempts'] = df['body_attempts'].expanding().mean().shift(1)
    df['avg_leg_strikes'] = df['leg_strikes'].expanding().mean().shift(1)
    df['avg_leg_attempts'] = df['leg_attempts'].expanding().mean().shift(1)
    df['avg_distance'] = df['distance'].expanding().mean().shift(1)
    df['avg_distance_attempts'] = df['distance_attempts'].expanding().mean().shift(1)
    df['avg_clinch_strikes'] = df['clinch_strikes'].expanding().mean().shift(1)
    df['avg_clinch_attempts'] = df['clinch_attempts'].expanding().mean().shift(1)
    df['avg_ground_strikes'] = df['ground_strikes'].expanding().mean().shift(1)
    df['avg_ground_attempts'] = df['ground_attempts'].expanding().mean().shift(1)
    
    return df

fighter_features = all_fights.groupby('fighter_name').apply(calculate_fighter_features, include_groups=False)

# Reset index first
fighter_features = fighter_features.reset_index()

# Separate red and blue features
red_features = fighter_features[fighter_features['corner'] == 'red'].copy()
blue_features = fighter_features[fighter_features['corner'] == 'blue'].copy()

# Add suffixes to feature columns
feature_cols = [col for col in fighter_features.columns if col.startswith('avg_')]

red_rename = {col: f'{col}_red' for col in feature_cols}
blue_rename = {col: f'{col}_blue' for col in feature_cols}

red_features = red_features.rename(columns=red_rename)
blue_features = blue_features.rename(columns=blue_rename)

# Merge back to original dataframe
df_filtered = df_filtered.merge(
    red_features[['fighter_name', 'event_date', 'opponent'] + list(red_rename.values())],
    left_on=['fighter_red', 'event_date', 'fighter_blue'],
    right_on=['fighter_name', 'event_date', 'opponent'],
    how='left'
).merge(
    blue_features[['fighter_name', 'event_date', 'opponent'] + list(blue_rename.values())],
    left_on=['fighter_blue', 'event_date', 'fighter_red'],
    right_on=['fighter_name', 'event_date', 'opponent'],
    how='left',
    suffixes=('', '_blue_temp')
)

# Clean up duplicate columns
df_filtered = df_filtered.drop(['fighter_name', 'fighter_name_blue_temp', 'opponent', 
                            'opponent_blue_temp'], axis=1, errors='ignore')

# Drop duplicates based on event_date, fighter_red, and fighter_blue
df_filtered = df_filtered.drop_duplicates(subset=['event_date', 'fighter_red', 'fighter_blue'])

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

Create new features

In [11]:
def create_fighter_record_features(df):
    """
    Create win/loss tracking features for UFC fighters while avoiding data leakage.
    
    Parameters:
    df: DataFrame with UFC fight data
    
    Returns:
    DataFrame with additional features for fighter records
    """
    # Create a copy to avoid modifying the original
    df_copy = df.copy()
    
    # Ensure event_date is datetime
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    
    # Sort by date to process fights chronologically
    df_copy = df_copy.sort_values('event_date').reset_index(drop=True)
    
    # Initialize tracking dictionaries for each fighter
    fighter_wins = {}
    fighter_losses = {}
    fighter_total_fights = {}
    
    # Initialize the new columns
    df_copy['wins_before_red'] = 0
    df_copy['losses_before_red'] = 0
    df_copy['total_fights_before_red'] = 0
    df_copy['wins_before_blue'] = 0
    df_copy['losses_before_blue'] = 0
    df_copy['total_fights_before_blue'] = 0
    
    # Process each fight chronologically
    for idx, row in df_copy.iterrows():
        red_fighter = row['fighter_red']
        blue_fighter = row['fighter_blue']
        winner = row['winner']
        
        # Get current records BEFORE this fight (to avoid data leakage)
        wins_before_red = fighter_wins.get(red_fighter, 0)
        losses_before_red = fighter_losses.get(red_fighter, 0)
        red_total_before = fighter_total_fights.get(red_fighter, 0)
        
        wins_before_blue = fighter_wins.get(blue_fighter, 0)
        losses_before_blue = fighter_losses.get(blue_fighter, 0)
        blue_total_before = fighter_total_fights.get(blue_fighter, 0)
        
        # Assign the records before this fight
        df_copy.at[idx, 'wins_before_red'] = wins_before_red
        df_copy.at[idx, 'losses_before_red'] = losses_before_red
        df_copy.at[idx, 'total_fights_before_red'] = red_total_before
        df_copy.at[idx, 'wins_before_blue'] = wins_before_blue
        df_copy.at[idx, 'losses_before_blue'] = losses_before_blue
        df_copy.at[idx, 'total_fights_before_blue'] = blue_total_before
        
        # Update records AFTER processing this fight
        # Red fighter
        if red_fighter not in fighter_wins:
            fighter_wins[red_fighter] = 0
            fighter_losses[red_fighter] = 0
            fighter_total_fights[red_fighter] = 0
        
        # Blue fighter
        if blue_fighter not in fighter_wins:
            fighter_wins[blue_fighter] = 0
            fighter_losses[blue_fighter] = 0
            fighter_total_fights[blue_fighter] = 0
        
        # Update based on fight result
        if winner == 1:  # Red fighter wins
            fighter_wins[red_fighter] += 1
            fighter_losses[blue_fighter] += 1
        else:  # Blue fighter wins
            fighter_wins[blue_fighter] += 1
            fighter_losses[red_fighter] += 1
        
        # Update total fights for both fighters
        fighter_total_fights[red_fighter] += 1
        fighter_total_fights[blue_fighter] += 1
    
    return df_copy

def create_recent_performance_features(df, recent_fights=3):
    """
    Create features based on recent fight performance
    This is more complex and requires tracking recent results
    """
    df_copy = df.copy()
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    df_copy = df_copy.sort_values('event_date').reset_index(drop=True)
    
    # Track recent results for each fighter
    fighter_recent_results = {}  # Will store list of recent results for each fighter
    
    # Initialize columns
    df_copy[f'wins_last_{recent_fights}_red'] = 0
    df_copy[f'wins_last_{recent_fights}_blue'] = 0
    df_copy[f'fights_last_{recent_fights}_red'] = 0
    df_copy[f'fights_last_{recent_fights}_blue'] = 0
    
    for idx, row in df_copy.iterrows():
        red_fighter = row['fighter_red']
        blue_fighter = row['fighter_blue']
        winner = row['winner']
        
        # Initialize if first time seeing fighter
        if red_fighter not in fighter_recent_results:
            fighter_recent_results[red_fighter] = []
        if blue_fighter not in fighter_recent_results:
            fighter_recent_results[blue_fighter] = []
        
        # Get recent form BEFORE this fight
        red_recent = fighter_recent_results[red_fighter][-recent_fights:]
        blue_recent = fighter_recent_results[blue_fighter][-recent_fights:]
        
        # Calculate recent performance
        df_copy.at[idx, f'wins_last_{recent_fights}_red'] = sum(red_recent)
        df_copy.at[idx, f'wins_last_{recent_fights}_blue'] = sum(blue_recent)
        df_copy.at[idx, f'fights_last_{recent_fights}_red'] = len(red_recent)
        df_copy.at[idx, f'fights_last_{recent_fights}_blue'] = len(blue_recent)
        
        # Update recent results AFTER processing this fight
        if winner == 1:  # Red wins
            fighter_recent_results[red_fighter].append(1)
            fighter_recent_results[blue_fighter].append(0)
        else:  # Blue wins
            fighter_recent_results[red_fighter].append(0)
            fighter_recent_results[blue_fighter].append(1)
    
    return df_copy

def create_win_ratio_record(df):
    """
    Create additional derived features from win/loss records
    """
    df_processed = df.copy()
    
    # Win percentage (handle division by zero)
    df_processed['win_pct_before_red'] = np.where(
        df_processed['total_fights_before_red'] > 0,
        (df_processed['wins_before_red'] / df_processed['total_fights_before_red']).round(3),
        0
    )
    
    df_processed['win_pct_before_blue'] = np.where(
        df_processed['total_fights_before_blue'] > 0,
        (df_processed['wins_before_blue'] / df_processed['total_fights_before_blue']).round(3),
        0
    )
        
    return df_processed

def create_days_since_debut_features(df):
    """
    Calculate days since each fighter's debut, avoiding data leakage.
    
    Parameters:
    df: DataFrame with UFC fight data (must be sorted by event_date)
    
    Returns:
    DataFrame with days_since_debut features added
    """
    df_copy = df.copy()
    
    # Ensure event_date is datetime and data is sorted
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    df_copy = df_copy.sort_values('event_date').reset_index(drop=True)
    
    # Track each fighter's debut date
    fighter_debut_dates = {}
    
    # Initialize the new columns
    df_copy['days_since_debut_red'] = 0
    df_copy['days_since_debut_blue'] = 0
    
    # Process each fight chronologically
    for idx, row in df_copy.iterrows():
        red_fighter = row['fighter_red']
        blue_fighter = row['fighter_blue']
        current_date = row['event_date']
        
        # Check if this is the fighter's debut (first time we see them)
        if red_fighter not in fighter_debut_dates:
            # This is red fighter's debut
            fighter_debut_dates[red_fighter] = current_date
            df_copy.at[idx, 'days_since_debut_red'] = 0
        else:
            # Calculate days since debut
            days_since = (current_date - fighter_debut_dates[red_fighter]).days
            df_copy.at[idx, 'days_since_debut_red'] = days_since
        
        if blue_fighter not in fighter_debut_dates:
            # This is blue fighter's debut
            fighter_debut_dates[blue_fighter] = current_date
            df_copy.at[idx, 'days_since_debut_blue'] = 0
        else:
            # Calculate days since debut
            days_since = (current_date - fighter_debut_dates[blue_fighter]).days
            df_copy.at[idx, 'days_since_debut_blue'] = days_since
    
    return df_copy

def create_days_since_last_win_features(df):
    """
    Calculate days since each fighter's last win, avoiding data leakage.
    
    Parameters:
    df: DataFrame with UFC fight data (must be sorted by event_date)
    
    Returns:
    DataFrame with days_since_last_win features added
    """
    df_copy = df.copy()
    
    # Ensure event_date is datetime and data is sorted
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    df_copy = df_copy.sort_values('event_date').reset_index(drop=True)
    
    # Track each fighter's last win date
    fighter_last_win_dates = {}
    
    # Initialize the new columns
    df_copy['days_since_last_win_red'] = np.nan  # NaN for fighters who never won
    df_copy['days_since_last_win_blue'] = np.nan
    
    # Process each fight chronologically
    for idx, row in df_copy.iterrows():
        red_fighter = row['fighter_red']
        blue_fighter = row['fighter_blue']
        current_date = row['event_date']
        winner = row['winner']
        
        # Calculate days since last win BEFORE this fight (to avoid data leakage)
        if red_fighter in fighter_last_win_dates:
            days_since = (current_date - fighter_last_win_dates[red_fighter]).days
            df_copy.at[idx, 'days_since_last_win_red'] = days_since
        else:
            # Fighter has never won before, or this is their first fight
            df_copy.at[idx, 'days_since_last_win_red'] = np.nan
        
        if blue_fighter in fighter_last_win_dates:
            days_since = (current_date - fighter_last_win_dates[blue_fighter]).days
            df_copy.at[idx, 'days_since_last_win_blue'] = days_since
        else:
            # Fighter has never won before, or this is their first fight
            df_copy.at[idx, 'days_since_last_win_blue'] = np.nan
        
        # Update last win dates AFTER processing this fight
        if winner == 1:  # Red fighter wins
            fighter_last_win_dates[red_fighter] = current_date
        elif winner == 0:  # Blue fighter wins
            fighter_last_win_dates[blue_fighter] = current_date
    
    # Add derived features
    
    # Binary indicators
    #df_copy['red_never_won'] = df_copy['days_since_last_win_red'].isna().astype(int)
    #df_copy['blue_never_won'] = df_copy['days_since_last_win_blue'].isna().astype(int)
    
    df_copy['recent_winner_red'] = (df_copy['days_since_last_win_red'] <= 365).astype(int)  # Won within last year
    df_copy['recent_winner_blue'] = (df_copy['days_since_last_win_blue'] <= 365).astype(int)
    
    # Handle NaN values for modeling (replace with a large number or separate indicator)
    # Replace NaN with a large number (e.g., 9999 days =~ 27.3 years) to indicate "never won"
    df_copy['days_since_last_win_red'] = df_copy['days_since_last_win_red'].fillna(9999)
    df_copy['days_since_last_win_blue'] = df_copy['days_since_last_win_blue'].fillna(9999)
    
    return df_copy

def create_wins_last_year_features(df):
    """
    Calculate wins in the last 365 days for each fighter, avoiding data leakage.
    
    Parameters:
    df: DataFrame with UFC fight data (must be sorted by event_date)
    
    Returns:
    DataFrame with wins_last_year features added
    """
    df_copy = df.copy()
    
    # Ensure event_date is datetime and data is sorted
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    df_copy = df_copy.sort_values('event_date').reset_index(drop=True)
    
    # Track fight history for each fighter (date, result pairs)
    fighter_fight_history = {}
    
    # Initialize the new columns
    df_copy['wins_last_365_days_red'] = 0
    df_copy['wins_last_365_days_blue'] = 0
    df_copy['fights_last_365_days_red'] = 0
    df_copy['fights_last_365_days_blue'] = 0
    
    # Process each fight chronologically
    for idx, row in df_copy.iterrows():
        red_fighter = row['fighter_red']
        blue_fighter = row['fighter_blue']
        current_date = row['event_date']
        winner = row['winner']
        
        # Initialize fighter history if first time seeing them
        if red_fighter not in fighter_fight_history:
            fighter_fight_history[red_fighter] = []
        if blue_fighter not in fighter_fight_history:
            fighter_fight_history[blue_fighter] = []
        
        # Calculate cutoff date for last 365 days (before current fight to avoid leakage)
        cutoff_date = current_date - timedelta(days=365)
        
        # Count wins and total fights in last 365 days for red fighter
        red_recent_fights = [
            fight for fight in fighter_fight_history[red_fighter] 
            if fight['date'] > cutoff_date
        ]
        red_wins_365 = sum(1 for fight in red_recent_fights if fight['won'])
        red_fights_365 = len(red_recent_fights)
        
        # Count wins and total fights in last 365 days for blue fighter
        blue_recent_fights = [
            fight for fight in fighter_fight_history[blue_fighter] 
            if fight['date'] > cutoff_date
        ]
        blue_wins_365 = sum(1 for fight in blue_recent_fights if fight['won'])
        blue_fights_365 = len(blue_recent_fights)
        
        # Assign the counts BEFORE this fight (avoiding data leakage)
        df_copy.at[idx, 'wins_last_365_days_red'] = red_wins_365
        df_copy.at[idx, 'wins_last_365_days_blue'] = blue_wins_365
        df_copy.at[idx, 'fights_last_365_days_red'] = red_fights_365
        df_copy.at[idx, 'fights_last_365_days_blue'] = blue_fights_365
        
        # Update fight history AFTER processing this fight
        # Add red fighter's result
        red_won = (winner == 1)
        fighter_fight_history[red_fighter].append({
            'date': current_date,
            'won': red_won
        })
        
        # Add blue fighter's result  
        blue_won = (winner == 0)
        fighter_fight_history[blue_fighter].append({
            'date': current_date,
            'won': blue_won
        })
    
    # Add derived features
    # Win rate in last 365 days
    '''
    df_copy['win_rate_last_365_days_red'] = np.where(
        df_copy['fights_last_365_days_red'] > 0,
        df_copy['wins_last_365_days_red'] / df_copy['fights_last_365_days_red'],
        0
    )
    
    df_copy['win_rate_last_365_days_blue'] = np.where(
        df_copy['fights_last_365_days_blue'] > 0,
        df_copy['wins_last_365_days_blue'] / df_copy['fights_last_365_days_blue'],
        0
    )
    '''

    df_copy['undefeated_last_year_red'] = (
        (df_copy['fights_last_365_days_red'] > 0) & 
        (df_copy['wins_last_365_days_red'] == df_copy['fights_last_365_days_red'])
    ).astype(int)
    
    df_copy['undefeated_last_year_blue'] = (
        (df_copy['fights_last_365_days_blue'] > 0) & 
        (df_copy['wins_last_365_days_blue'] == df_copy['fights_last_365_days_blue'])
    ).astype(int)
    
    return df_copy

def create_win_lose_streak_features(df):
    """
    Calculate current win/lose streaks for each fighter, avoiding data leakage.
    
    Parameters:
    df: DataFrame with UFC fight data (must be sorted by event_date)
    
    Returns:
    DataFrame with win/lose streak features added
    """
    df_copy = df.copy()
    
    # Ensure event_date is datetime and data is sorted
    df_copy['event_date'] = pd.to_datetime(df_copy['event_date'])
    df_copy = df_copy.sort_values('event_date').reset_index(drop=True)
    
    # Track streak information for each fighter
    fighter_streaks = {}
    
    # Initialize the new columns
    df_copy['win_streak_red'] = 0
    df_copy['win_streak_blue'] = 0
    df_copy['lose_streak_red'] = 0
    df_copy['lose_streak_blue'] = 0
    
    # Process each fight chronologically
    for idx, row in df_copy.iterrows():
        red_fighter = row['fighter_red']
        blue_fighter = row['fighter_blue']
        winner = row['winner']
        
        # Initialize fighter streak tracking if first time seeing them
        if red_fighter not in fighter_streaks:
            fighter_streaks[red_fighter] = {
                'current_win_streak': 0,
                'current_lose_streak': 0,
                'last_result': None  # 'win', 'loss', or None
            }
        
        if blue_fighter not in fighter_streaks:
            fighter_streaks[blue_fighter] = {
                'current_win_streak': 0,
                'current_lose_streak': 0,
                'last_result': None
            }
        
        # Get current streaks BEFORE this fight (to avoid data leakage)
        win_streak_red = fighter_streaks[red_fighter]['current_win_streak']
        lose_streak_red = fighter_streaks[red_fighter]['current_lose_streak']
        win_streak_blue = fighter_streaks[blue_fighter]['current_win_streak']
        lose_streak_blue = fighter_streaks[blue_fighter]['current_lose_streak']
        
        # Assign streaks before this fight
        df_copy.at[idx, 'win_streak_red'] = win_streak_red
        df_copy.at[idx, 'lose_streak_red'] = lose_streak_red
        df_copy.at[idx, 'win_streak_blue'] = win_streak_blue
        df_copy.at[idx, 'lose_streak_blue'] = lose_streak_blue
        
        # Update streaks AFTER processing this fight
        if winner == 1:  # Red fighter wins
            # Red fighter wins - update win streak, reset lose streak
            fighter_streaks[red_fighter]['current_win_streak'] += 1
            fighter_streaks[red_fighter]['current_lose_streak'] = 0
            fighter_streaks[red_fighter]['last_result'] = 'win'
            
            # Blue fighter loses - update lose streak, reset win streak
            fighter_streaks[blue_fighter]['current_lose_streak'] += 1
            fighter_streaks[blue_fighter]['current_win_streak'] = 0
            fighter_streaks[blue_fighter]['last_result'] = 'loss'
            
        elif winner == 0:  # Blue fighter wins
            # Blue fighter wins - update win streak, reset lose streak
            fighter_streaks[blue_fighter]['current_win_streak'] += 1
            fighter_streaks[blue_fighter]['current_lose_streak'] = 0
            fighter_streaks[blue_fighter]['last_result'] = 'win'
            
            # Red fighter loses - update lose streak, reset win streak
            fighter_streaks[red_fighter]['current_lose_streak'] += 1
            fighter_streaks[red_fighter]['current_win_streak'] = 0
            fighter_streaks[red_fighter]['last_result'] = 'loss'
    
    # Add derived features
    
    # Momentum indicators
    df_copy['on_win_streak_red'] = (df_copy['win_streak_red'] >= 1).astype(int)
    df_copy['on_win_streak_blue'] = (df_copy['win_streak_blue'] >= 1).astype(int)
    #df_copy['red_on_lose_streak'] = (df_copy['lose_streak_red'] >= 1).astype(int)
    #df_copy['blue_on_lose_streak'] = (df_copy['lose_streak_blue'] >= 1).astype(int)
    
    # Long streak indicators (3+ wins/losses)
    df_copy['long_win_streak_red'] = (df_copy['win_streak_red'] >= 3).astype(int)
    df_copy['long_win_streak_blue'] = (df_copy['win_streak_blue'] >= 3).astype(int)
    #df_copy['red_long_lose_streak'] = (df_copy['lose_streak_red'] >= 3).astype(int)
    #df_copy['blue_long_lose_streak'] = (df_copy['lose_streak_blue'] >= 3).astype(int)
    
    return df_copy

# Example usage with your dataset
def process_ufc_data(df):
    """
    Complete pipeline to add all fighter record features
    """
    print("Processing UFC data for feature engineering...")
    print(f"Original dataset shape: {df.shape}")
    
    # Step 1: Create win/loss features
    df_with_records = create_fighter_record_features(df)
    print("✓ Added win/loss tracking features")

    # Step 2: Add a win ratio
    df_processed = create_win_ratio_record(df_with_records)
    print("✓ Added win ratio features")
    
    # Step 3: Add recent form features
    df_processed = create_recent_performance_features(df_processed, recent_fights=3)
    print("✓ Added recent performance features")

    # Step 4: Days since debut
    df_processed = create_days_since_debut_features(df_processed)
    print("✓ Added days since debut features")

    # Step 5: Add days since last win
    df_processed = create_days_since_last_win_features(df_processed)
    print("✓ Added days since last win/loss features")

    # Step 6: Add wins in last calendar year
    df_processed = create_wins_last_year_features(df_processed)
    print("✓ Added calendar year features")

    # Step 7: Add win/loss streak
    df_final = create_win_lose_streak_features(df_processed)
    print("✓ Added win/loss streak features")
    
    print(f"Final dataset shape: {df_final.shape}")
    print(f"Added {df_final.shape[1] - df.shape[1]} new features")
    
    return df_final

df_processed = process_ufc_data(df_filtered)

Processing UFC data for feature engineering...
Original dataset shape: (33, 115)
✓ Added win/loss tracking features
✓ Added win ratio features
✓ Added recent performance features
✓ Added days since debut features
✓ Added days since last win/loss features
✓ Added calendar year features
✓ Added win/loss streak features
Final dataset shape: (33, 147)
Added 32 new features


Temporal Feature Engineering

In [12]:
def get_fighter_stats(df: pd.DataFrame, fighter_name: str, fight_index: int) -> Dict:
    """
    Get the stats for a specific fighter in a specific fight, regardless of red/blue corner
    """
    try:
        row = df.iloc[fight_index]
        
        if row['fighter_red'] == fighter_name:
            corner = 'red'
        elif row['fighter_blue'] == fighter_name:
            corner = 'blue'
        else:
            return None
        
        # Define the key stats we want to extract (focusing on most important ones)
        stat_names = [
            'sig_strikes', 'sig_attempts', 'knockdowns', 'takedowns', 'takedown_attempts', 
            'total_strikes', 'total_strikes_attempts','sub_attempts', 'control_time',
            'head_strikes', 'head_attempts'
        ]
        
        stats = {}
        for stat in stat_names:
            col_name = f"{stat}_{corner}"
            stats[stat] = row.get(col_name, np.nan)
        
        # Also get fight outcome and method
        stats['won'] = 1 if row['winner'] == fighter_name else 0
        stats['win_method'] = row.get('win_method', np.nan)
        stats['round'] = row.get('round', np.nan)
        stats['event_date'] = row.get('event_date')
    except (IndexError, KeyError) as e:
        print(f"Error getting stats for {fighter_name} at index {fight_index}: {e}")
        return None
    
    return stats

def calculate_essential_ema_features(df: pd.DataFrame, alpha: float = 0.3) -> pd.DataFrame:
    """
    Calculate EMAs for only the most important performance metrics
    Features end with _red or _blue
    """
    result_df = df.copy()
    
    # Get all unique fighters
    all_fighters = set(df['fighter_red'].dropna().unique()) | set(df['fighter_blue'].dropna().unique())
    
    # Focus on essential stats only
    essential_stats = ['sig_strikes', 'sig_attempts', 'knockdowns', 'takedowns', 'takedown_attempts', 
            'total_strikes', 'total_strikes_attempts', 'head_strikes', 'head_attempts']
    
    # Initialize EMA columns
    for corner in ['red', 'blue']:
        for stat in essential_stats:
            result_df[f"{stat}_ema_{corner}"] = np.nan
            if 'attempts' not in stat:
                result_df[f"{stat}_success_rate_ema_{corner}"] = np.nan
    
    # Calculate EMAs for each fighter
    for fighter in all_fighters:
        if pd.isna(fighter):
            continue
        
        # Get all fights for this fighter in chronological order
        fighter_fights = []
        for idx, row in df.iterrows():
            if row['fighter_red'] == fighter or row['fighter_blue'] == fighter:
                stats = get_fighter_stats(df, fighter, df.index.get_loc(idx))
                if stats:
                    fighter_fights.append({
                        'index': idx,
                        'corner': 'red' if row['fighter_red'] == fighter else 'blue',
                        'stats': stats
                    })
        
        if len(fighter_fights) <= 1:
            continue
        
        # Calculate EMAs for this fighter
        fighter_emas = {}
        
        for i, fight in enumerate(fighter_fights):
            if i == 0:
                # First fight - initialize EMAs with current values
                for stat in essential_stats:
                    if pd.notna(fight['stats'][stat]):
                        fighter_emas[f"{stat}_ema"] = fight['stats'][stat]
                        
                        # Initialize success rate EMAs
                        if 'attempts' not in stat and f"{stat}_attempts" in fight['stats']:
                            attempts = fight['stats'][f"{stat}_attempts"]
                            if attempts > 0:
                                fighter_emas[f"{stat}_success_rate_ema"] = fight['stats'][stat] / attempts
                            else:
                                fighter_emas[f"{stat}_success_rate_ema"] = 0
                continue
            
            # Update EMAs using previous fight's values (avoid data leakage)
            prev_fight = fighter_fights[i-1]
            current_corner = fight['corner']
            
            for stat in essential_stats:
                ema_key = f"{stat}_ema"
                success_rate_key = f"{stat}_success_rate_ema"
                
                # Update volume EMA
                if ema_key in fighter_emas and pd.notna(prev_fight['stats'][stat]):
                    new_ema = alpha * prev_fight['stats'][stat] + (1 - alpha) * fighter_emas[ema_key]
                    fighter_emas[ema_key] = new_ema
                    result_df.loc[fight['index'], f"{stat}_ema_{current_corner}"] = fighter_emas[ema_key]
                
                # Update success rate EMA
                if 'attempts' not in stat and f"{stat}_attempts" in prev_fight['stats']:
                    prev_attempts = prev_fight['stats'][f"{stat}_attempts"]
                    if prev_attempts > 0:
                        prev_success_rate = prev_fight['stats'][stat] / prev_attempts
                        
                        if success_rate_key in fighter_emas:
                            new_success_ema = alpha * prev_success_rate + (1 - alpha) * fighter_emas[success_rate_key]
                            fighter_emas[success_rate_key] = new_success_ema
                            result_df.loc[fight['index'], f"{stat}_success_rate_ema_{current_corner}"] = fighter_emas[success_rate_key]
    
    return result_df

def calculate_essential_rolling_features(df: pd.DataFrame, window: int = 5) -> pd.DataFrame:
    """
    Calculate rolling features for essential metrics only, using 5-fight window
    Features end with _red or _blue
    """
    result_df = df.copy()
    
    # Get all unique fighters
    all_fighters = set(df['fighter_red'].dropna().unique()) | set(df['fighter_blue'].dropna().unique())
    
    # Essential stats only
    essential_stats = ['sig_strikes', 'sig_attempts',
                    'takedowns', 'takedown_attempts', 'control_time',
                    'knockdowns', 'total_strikes', 'total_strikes_attempts']
    
    # Initialize rolling columns
    for corner in ['red', 'blue']:
        for stat in essential_stats:
            result_df[f"{stat}_roll_{window}_{corner}"] = np.nan
            if stat in ['sig_strikes', 'takedowns']:  # Only for stats with success rates
                result_df[f"{stat}_success_rate_roll_{window}_{corner}"] = np.nan
    
    # Calculate rolling features for each fighter
    for fighter in all_fighters:
        if pd.isna(fighter):
            continue
        
        # Get all fights for this fighter
        fighter_fights = []
        for idx, row in df.iterrows():
            if row['fighter_red'] == fighter or row['fighter_blue'] == fighter:
                stats = get_fighter_stats(df, fighter, df.index.get_loc(idx))
                if stats:
                    fighter_fights.append({
                        'index': idx,
                        'corner': 'red' if row['fighter_red'] == fighter else 'blue',
                        'stats': stats
                    })
        
        if len(fighter_fights) <= 1:
            continue
        
        # Create series for each stat for this fighter
        fighter_stats_series = {}
        indices = []
        
        for fight in fighter_fights:
            indices.append(fight['index'])
            for stat in essential_stats:
                if stat not in fighter_stats_series:
                    fighter_stats_series[stat] = []
                fighter_stats_series[stat].append(fight['stats'][stat])
        
        # Convert to pandas series
        for stat in essential_stats:
            fighter_stats_series[stat] = pd.Series(fighter_stats_series[stat], index=indices)
        
        # Calculate rolling features
        for stat in essential_stats:
            if stat in fighter_stats_series:
                series = fighter_stats_series[stat]
                
                # Rolling mean (shifted by 1 to avoid data leakage)
                rolling_mean = series.rolling(window=window, min_periods=1).mean().shift(1)
                
                # Apply to correct corner for each fight
                for fight in fighter_fights:
                    idx = fight['index']
                    corner = fight['corner']
                    
                    if idx in rolling_mean.index:
                        result_df.loc[idx, f"{stat}_roll_{window}_{corner}"] = rolling_mean.loc[idx]
                
                # Calculate success rate rolling features for applicable stats
                if stat == 'sig_strikes':
                    attempts_stat = 'sig_attempts'
                elif stat == 'takedowns':
                    attempts_stat = 'takedown_attempts'
                else:
                    continue
                
                if attempts_stat in fighter_stats_series:
                    success_series = fighter_stats_series[stat] / (fighter_stats_series[attempts_stat] + 1e-8)
                    rolling_success = success_series.rolling(window=window, min_periods=1).mean().shift(1)
                    
                    for fight in fighter_fights:
                        idx = fight['index']
                        corner = fight['corner']
                        
                        if idx in rolling_success.index:
                            result_df.loc[idx, f"{stat}_success_rate_roll_{window}_{corner}"] = rolling_success.loc[idx]
    
    return result_df

def calculate_essential_momentum_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate only the most valuable momentum features that aren't redundant
    with existing win/loss tracking. Features end with _red or _blue
    """
    result_df = df.copy()
    
    # Get all unique fighters
    all_fighters = set(df['fighter_red'].dropna().unique()) | set(df['fighter_blue'].dropna().unique())
    
    # Initialize momentum columns (only the essential ones)
    for corner in ['red', 'blue']:
        result_df[f'performance_trend_{corner}'] = np.nan          # Shows trajectory
        result_df[f'finish_momentum_{corner}'] = np.nan           # How they win
        result_df[f'dominance_momentum_{corner}'] = np.nan        # Quality of wins
    
    for fighter in all_fighters:
        if pd.isna(fighter):
            continue
        
        # Get all fights for this fighter
        fighter_fights = []
        for idx, row in df.iterrows():
            if row['fighter_red'] == fighter or row['fighter_blue'] == fighter:
                stats = get_fighter_stats(df, fighter, df.index.get_loc(idx))
                if stats:
                    fighter_fights.append({
                        'index': idx,
                        'corner': 'red' if row['fighter_red'] == fighter else 'blue',
                        'stats': stats
                    })
        
        if len(fighter_fights) <= 2:  # Need at least 3 fights for trends
            continue
        
        # Extract win/loss sequence
        win_sequence = [fight['stats']['won'] for fight in fighter_fights]
        
        # 1. Performance Trend (linear trend of recent results)
        performance_trend = []
        for i in range(len(win_sequence)):
            if i < 2:
                performance_trend.append(np.nan)
            else:
                recent_results = win_sequence[max(0, i-4):i]  # Last 5 fights before current
                if len(recent_results) >= 3:
                    x = np.arange(len(recent_results))
                    try:
                        trend = np.polyfit(x, recent_results, 1)[0]  # Slope
                        performance_trend.append(trend)
                    except:
                        performance_trend.append(np.nan)
                else:
                    performance_trend.append(np.nan)
        
        # 2. Finish Momentum (recent finish rate)
        finish_momentum = []
        for i, fight in enumerate(fighter_fights):
            if i == 0:
                finish_momentum.append(np.nan)
                continue
            
            # Look at previous fights
            recent_fights = fighter_fights[max(0, i-5):i]  # Last 5 fights before current
            wins = [f for f in recent_fights if f['stats']['won'] == 1]
            
            if len(wins) > 0:
                finishes = [f for f in wins if pd.notna(f['stats']['win_method']) and 
                        f['stats']['win_method'] in ['Submission', 'KO/TKO', 'TKO']]
                finish_rate = len(finishes) / len(wins)
                finish_momentum.append(finish_rate)
            else:
                finish_momentum.append(0.0)
        
        # 3. Dominance Momentum (how decisively they win)
        dominance_momentum = []
        for i, fight in enumerate(fighter_fights):
            if i == 0:
                dominance_momentum.append(np.nan)
                continue
            
            # Look at previous fights
            recent_fights = fighter_fights[max(0, i-3):i]  # Last 3 fights before current
            dominance_scores = []
            
            for f in recent_fights:
                if f['stats']['won'] == 1:
                    # Score based on how they won
                    if pd.notna(f['stats']['round']) and f['stats']['round'] == 1:
                        dominance_scores.append(1.0)  # First round finish
                    elif pd.notna(f['stats']['win_method']) and f['stats']['win_method'] in ['Submission', 'KO/TKO', 'TKO']:
                        dominance_scores.append(0.8)  # Later round finish
                    else:
                        dominance_scores.append(0.6)  # Decision win
                else:
                    dominance_scores.append(0.0)  # Loss
            
            if len(dominance_scores) > 0:
                dominance_momentum.append(np.mean(dominance_scores))
            else:
                dominance_momentum.append(np.nan)
        
        # Apply momentum features to the dataframe
        for i, fight in enumerate(fighter_fights):
            idx = fight['index']
            corner = fight['corner']
            
            result_df.loc[idx, f'performance_trend_{corner}'] = performance_trend[i]
            result_df.loc[idx, f'finish_momentum_{corner}'] = finish_momentum[i]
            result_df.loc[idx, f'dominance_momentum_{corner}'] = dominance_momentum[i]
    
    return result_df

def engineer_essential_temporal_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Main function to apply essential temporal feature engineering only
    All features end with _red or _blue
    """
    print("Calculating essential EMA features...")
    df = calculate_essential_ema_features(df, alpha=0.3)
    
    print("Calculating essential rolling window features (5-fight window)...")
    df = calculate_essential_rolling_features(df, window=5)
    
    print("Calculating essential momentum features...")
    df = calculate_essential_momentum_features(df)
    
    print("Essential feature engineering complete!")
    
    # Print summary of new features
    temporal_cols = [col for col in df.columns if any(x in col for x in 
                    ['_ema_', '_roll_', '_momentum_', '_trend_'])]
    print(f"Added {len(temporal_cols)} new temporal features:")
    
    feature_types = {
        'EMA': [col for col in temporal_cols if '_ema_' in col],
        'Rolling': [col for col in temporal_cols if '_roll_' in col], 
        'Momentum': [col for col in temporal_cols if '_momentum_' in col or '_trend_' in col]
    }
    
    for ftype, cols in feature_types.items():
        print(f"  {ftype}: {len(cols)} features")
    
    return df

def handle_temporal_nans(df: pd.DataFrame) -> pd.DataFrame:
    """
    Handle NaN values in temporal features based on their specific context
    """
    df_filled = df.copy()
    
    # Get all temporal columns
    temporal_cols = [col for col in df.columns if any(x in col for x in 
                    ['_ema_', '_roll_', '_momentum_', '_trend_'])]
    
    print(f"Handling NaN values in {len(temporal_cols)} temporal features...")
    
    # Get all fighters
    all_fighters = set(df['fighter_red'].dropna().unique()) | set(df['fighter_blue'].dropna().unique())
    
    # 1. Handle early career NaNs (fighters with insufficient history)
    print("1. Filling early career NaNs...")
    for fighter in all_fighters:
        if pd.isna(fighter):
            continue
            
        fighter_fights = df_filled[
            (df_filled['fighter_red'] == fighter) | 
            (df_filled['fighter_blue'] == fighter)
        ].sort_values('event_date')
        
        if len(fighter_fights) <= 1:
            continue
        
        # Forward fill from first valid value for each temporal feature
        for col in temporal_cols:
            if col in fighter_fights.columns:
                fighter_values = fighter_fights[col].copy()
                first_valid_idx = fighter_values.first_valid_index()
                
                if first_valid_idx is not None:
                    first_valid_value = fighter_values.loc[first_valid_idx]
                    # Fill earlier NaNs with first valid value
                    early_fights = fighter_fights[fighter_fights.index < first_valid_idx]
                    for early_idx in early_fights.index:
                        if pd.isna(df_filled.loc[early_idx, col]):
                            df_filled.loc[early_idx, col] = first_valid_value
    
    # 2. Fill momentum features with contextually meaningful defaults
    print("2. Filling momentum feature NaNs...")
    
    # Performance trends: 0 (no trend)
    trend_cols = [col for col in temporal_cols if 'trend' in col]
    for col in trend_cols:
        df_filled[col] = df_filled[col].fillna(0.0)
    
    # Finish momentum: 0 (no finishing history)
    finish_cols = [col for col in temporal_cols if 'finish_momentum' in col]
    for col in finish_cols:
        df_filled[col] = df_filled[col].fillna(0.0)
    
    # Dominance momentum: Use conservative estimate (0.3 = low dominance)
    dominance_cols = [col for col in temporal_cols if 'dominance_momentum' in col]
    for col in dominance_cols:
        df_filled[col] = df_filled[col].fillna(0.3)
    
    # 3. Fill success rate features with population medians
    print("3. Filling success rate NaNs...")
    success_rate_cols = [col for col in temporal_cols if 'success_rate' in col]
    for col in success_rate_cols:
        if col in df_filled.columns:
            # Use dropna() here too to avoid warnings
            non_nan_pop = df_filled[col].dropna()
            if len(non_nan_pop) > 0:
                population_median = non_nan_pop.median()
                df_filled[col] = df_filled[col].fillna(population_median)
            else:
                # Fallback defaults based on typical MMA stats
                if 'sig_strikes' in col:
                    df_filled[col] = df_filled[col].fillna(0.45)  # ~45% striking accuracy
                elif 'takedowns' in col:
                    df_filled[col] = df_filled[col].fillna(0.35)  # ~35% takedown success
                else:
                    df_filled[col] = df_filled[col].fillna(0.0)

    # 4. Fill remaining EMA and rolling features with fighter-specific medians
    print("4. Filling remaining EMA and rolling NaNs...")
    remaining_cols = [col for col in temporal_cols if col not in (trend_cols + finish_cols + dominance_cols + success_rate_cols)]

    for fighter in all_fighters:
        if pd.isna(fighter):
            continue
            
        fighter_mask = (df_filled['fighter_red'] == fighter) | (df_filled['fighter_blue'] == fighter)
        fighter_data = df_filled[fighter_mask]
        
        if len(fighter_data) <= 1:
            continue
        
        for col in remaining_cols:
            if col in fighter_data.columns:
                # Use dropna() to avoid empty slice warnings
                non_nan_values = fighter_data[col].dropna()
                
                if len(non_nan_values) > 0:
                    fighter_median = non_nan_values.median()
                    df_filled.loc[fighter_mask & df_filled[col].isna(), col] = fighter_median
                else:
                    # No valid values for this fighter, use population median
                    pop_non_nan = df_filled[col].dropna()
                    if len(pop_non_nan) > 0:
                        pop_median = pop_non_nan.median()
                        df_filled.loc[fighter_mask & df_filled[col].isna(), col] = pop_median
                    else:
                        # Entire column is NaN, use 0 as fallback
                        df_filled.loc[fighter_mask & df_filled[col].isna(), col] = 0.0
    
    # 5. Final cleanup
    print("5. Final cleanup...")
    remaining_nans = df_filled[temporal_cols].isna().sum().sum()
    if remaining_nans > 0:
        print(f"   Filling {remaining_nans} remaining NaNs with 0...")
        for col in temporal_cols:
            df_filled[col] = df_filled[col].fillna(0.0)
    
    # Validation
    final_nans = df_filled[temporal_cols].isna().sum().sum()
    print(f"\nNaN Filling Complete!")
    print(f"All {len(temporal_cols)} temporal features now NaN-free")
    print(f"{len(df_filled)} fights ready for ML training")
    
    return df_filled

# Example usage
if __name__ == "__main__":
    print("Essential UFC Feature Engineering Ready!")

Essential UFC Feature Engineering Ready!


In [13]:
# Add essential temporal features
df_temporal_features = engineer_essential_temporal_features(df_processed)

# Handle NaN values appropriately
df_temporal_features_clean = handle_temporal_nans(df_temporal_features)

Calculating essential EMA features...
Calculating essential rolling window features (5-fight window)...
Calculating essential momentum features...
Essential feature engineering complete!
Added 54 new temporal features:
  EMA: 28 features
  Rolling: 20 features
  Momentum: 6 features
Handling NaN values in 54 temporal features...
1. Filling early career NaNs...
2. Filling momentum feature NaNs...
3. Filling success rate NaNs...
4. Filling remaining EMA and rolling NaNs...
5. Final cleanup...

NaN Filling Complete!
All 54 temporal features now NaN-free
33 fights ready for ML training


In [14]:
df_temporal_features_clean = df_temporal_features_clean.tail(1)

In [15]:
# Convert event_date
df_temporal_features_clean['year'] = pd.to_datetime(df_temporal_features_clean['event_date']).dt.year
df_temporal_features_clean['month'] = pd.to_datetime(df_temporal_features_clean['event_date']).dt.month  
df_temporal_features_clean['day_of_week'] = pd.to_datetime(df_temporal_features_clean['event_date']).dt.dayofweek

In [16]:
# Load encoder
with open("../models/encoder_stance.pkl", "rb") as f:
    encoder_stance = pickle.load(f)

# Encode stance_red and stance_blue separately to preserve naming
encoder_stance = OneHotEncoder(sparse_output=False, drop='first')

encoder_stance.fit(df_temporal_features_clean[['stance_red']])

# Transform red stance
stance_red_encoded = encoder_stance.transform(df_temporal_features_clean[['stance_red']])
stance_red_names = [name.replace('stance_red_', '') + '_red'
                    for name in encoder_stance.get_feature_names_out(['stance_red'])]

# Transform blue stance (rename column temporarily to match fit)
stance_blue_encoded = encoder_stance.transform(
    df_temporal_features_clean[['stance_blue']].rename(columns={'stance_blue': 'stance_red'})
)
stance_blue_names = [name.replace('stance_red_', '') + '_blue'
                    for name in encoder_stance.get_feature_names_out(['stance_red'])]

# Combine all encoded features
all_encoded = np.concatenate([stance_red_encoded, stance_blue_encoded], axis=1)
all_feature_names = stance_red_names + stance_blue_names

# Create final DataFrame
encoded_df = pd.DataFrame(all_encoded, columns=all_feature_names, index=df_temporal_features_clean.index)

# Concatenate and drop originals
df_temporal_features_clean = pd.concat([df_temporal_features_clean.drop(['stance_red', 'stance_blue'], axis=1), encoded_df], axis=1)

# Drop columns
df_temporal_features_clean.drop(columns=['event_date', 'event_name', 'weight_class', 'win_method', 'fighter_blue', 'fighter_red', 'octagon_debut_blue', 'octagon_debut_red',
                                        'knockdowns_red','knockdowns_blue','sig_attempts_red','sig_attempts_blue','sig_strikes_red','sig_strikes_blue','total_strikes_attempts_red',
                                        'total_strikes_attempts_blue','total_strikes_red','total_strikes_blue','sub_attempts_red','sub_attempts_blue','takedowns_red',
                                        'takedowns_blue','takedown_attempts_red','takedown_attempts_blue','control_time_red','control_time_blue','head_strikes_red',
                                        'head_strikes_blue','head_attempts_red','head_attempts_blue','body_strikes_red','body_strikes_blue','body_attempts_red',
                                        'body_attempts_blue','leg_strikes_red','leg_strikes_blue','leg_attempts_red','leg_attempts_blue','distance_red','distance_blue',
                                        'distance_attempts_red','distance_attempts_blue','clinch_strikes_red','clinch_strikes_blue','clinch_attempts_red','clinch_attempts_blue',
                                        'ground_strikes_red','ground_strikes_blue','ground_attempts_red','ground_attempts_blue'], inplace=True)

# Calculate differences
# Find numeric _blue columns
blue_cols = [col for col in df_temporal_features_clean.columns 
            if col.endswith('_blue') and pd.api.types.is_numeric_dtype(df_temporal_features_clean[col])]

# Create all difference columns at once
diff_data = {}
cols_to_drop = []

for blue_col in blue_cols:
    red_col = blue_col.replace('_blue', '_red')
    if red_col in df_temporal_features_clean.columns and pd.api.types.is_numeric_dtype(df_temporal_features_clean[red_col]):
        diff_col = blue_col.replace('_blue', '_diff')
        diff_data[diff_col] = df_temporal_features_clean[blue_col] - df_temporal_features_clean[red_col]
        cols_to_drop.extend([blue_col, red_col])

# Create difference DataFrame and concatenate
if diff_data:
    diff_df = pd.DataFrame(diff_data, index=df_temporal_features_clean.index)
    
    # Drop old columns and add new ones in one operation
    df_temporal_features_clean = df_temporal_features_clean.drop(cols_to_drop, axis=1)
    df_temporal_features_clean = pd.concat([df_temporal_features_clean, diff_df], axis=1)

# Defragment the DataFrame
df_temporal_features_clean = df_temporal_features_clean.copy()

ValueError: Found unknown categories ['Orthodox'] in column 0 during transform

In [ ]:
df_temporal_features_clean

,round,time,winner,year,month,day_of_week,height_diff,weight_diff,reach_diff,leg_reach_diff,sig_strikes_landed_per_minute_diff,sig_strikes_absorbed_per_minute_diff,takedowns_avg_diff,submission_avg_diff,knockdown_avg_diff,fight_time_avg_diff,avg_knockdowns_diff,avg_sig_attempts_diff,avg_sig_strikes_diff,avg_total_strikes_attempts_diff,avg_total_strikes_diff,avg_sub_attempts_diff,avg_takedowns_diff,avg_takedown_attempts_diff,avg_head_strikes_diff,avg_head_attempts_diff,avg_body_strikes_diff,avg_body_attempts_diff,avg_leg_strikes_diff,avg_leg_attempts_diff,avg_distance_diff,avg_distance_attempts_diff,avg_clinch_strikes_diff,avg_clinch_attempts_diff,avg_ground_strikes_diff,avg_ground_attempts_diff,wins_before_diff,losses_before_diff,total_fights_before_diff,win_pct_before_diff,wins_last_3_diff,fights_last_3_diff,days_since_debut_diff,days_since_last_win_diff,recent_winner_diff,wins_last_365_days_diff,fights_last_365_days_diff,undefeated_last_year_diff,win_streak_diff,lose_streak_diff,on_win_streak_diff,long_win_streak_diff,sig_strikes_ema_diff,sig_strikes_success_rate_ema_diff,sig_attempts_ema_diff,knockdowns_ema_diff,knockdowns_success_rate_ema_diff,takedowns_ema_diff,takedowns_success_rate_ema_diff,takedown_attempts_ema_diff,total_strikes_ema_diff,total_strikes_success_rate_ema_diff,total_strikes_attempts_ema_diff,head_strikes_ema_diff,head_strikes_success_rate_ema_diff,head_attempts_ema_diff,sig_strikes_roll_5_diff,sig_strikes_success_rate_roll_5_diff,sig_attempts_roll_5_diff,takedowns_roll_5_diff,takedowns_success_rate_roll_5_diff,takedown_attempts_roll_5_diff,control_time_roll_5_diff,knockdowns_roll_5_diff,total_strikes_roll_5_diff,total_strikes_attempts_roll_5_diff,performance_trend_diff,finish_momentum_diff,dominance_momentum_diff
20,NaN,NaN,NaN,2025,9,6,3.0,3.0,4.0,1.0,0.0,1.0,0.79,0.36,-0.34,194.0,-0.631579,113.947368,44.684211,121.263158,51.0,0.315789,0.684211,1.0,33.263158,99.0,11.578947,14.263158,-0.157895,0.684211,49.526316,119.210526,-0.105263,1.368421,-4.736842,-6.631579,11,7,18,-0.368,1,2,3920,35.0,0,1,1,0,1,0,0,0,50.846677,0.0,116.34807,-0.89862,0.0,0.277427,0.0,0.210675,60.144809,-0.162927,127.503277,41.995128,0.0,105.442675,21.8,-0.235767,64.2,0.2,0.1,0.2,8.2,-0.8,28.8,73.0,0.0,0.0,-0.3


In [ ]:
df_temporal_features_clean.loc[df_temporal_features_clean.index[-1], 'round'] = 2.0
df_temporal_features_clean.loc[df_temporal_features_clean.index[-1], 'time'] = 232.0

Feature Selection

In [ ]:
# Use features selceted from training
# Load top features
top_features_df = pd.read_csv('../data/notebooks/top_features.csv')
top_features = top_features_df['feature'].tolist()  # convert to list

# Keep only those columns in your DataFrame
X_selected = df_temporal_features_clean[top_features]

In [ ]:
X_selected

,year,fight_time_avg_diff,days_since_debut_diff,win_pct_before_diff,sig_strikes_landed_per_minute_diff,knockdown_avg_diff,takedowns_avg_diff,avg_ground_attempts_diff,days_since_last_win_diff,control_time_roll_5_diff,avg_sub_attempts_diff,losses_before_diff,takedowns_ema_diff,avg_ground_strikes_diff,avg_takedowns_diff,avg_clinch_attempts_diff,avg_leg_attempts_diff,takedowns_success_rate_roll_5_diff,avg_body_strikes_diff,avg_takedown_attempts_diff,submission_avg_diff,avg_body_attempts_diff,takedown_attempts_ema_diff,weight_diff,avg_leg_strikes_diff,avg_clinch_strikes_diff,reach_diff,avg_head_strikes_diff,knockdowns_ema_diff,avg_head_attempts_diff,head_strikes_ema_diff,takedown_attempts_roll_5_diff,avg_knockdowns_diff,sig_attempts_roll_5_diff,head_attempts_ema_diff,sig_strikes_ema_diff,sig_strikes_roll_5_diff,avg_total_strikes_diff,avg_distance_diff,leg_reach_diff
20,2025,194.0,3920,-0.368,0.0,-0.34,0.79,-6.631579,35.0,8.2,0.315789,7,0.277427,-4.736842,0.684211,1.368421,0.684211,0.1,11.578947,1.0,0.36,14.263158,0.210675,3.0,-0.157895,-0.105263,4.0,33.263158,-0.89862,99.0,41.995128,0.2,-0.631579,64.2,105.442675,50.846677,21.8,51.0,49.526316,1.0


Traning

In [ ]:
# Load the scaler
with open("../models/scaler.pkl", "rb") as f:
    scaler_loaded = pickle.load(f)

# Transform new data
X_scaled  = scaler_loaded.transform(X_selected)
# Convert to dictionary (column name -> scaled value)
# Assuming you have a single row; if multiple rows, you can iterate
fighter_differences = dict(zip(X_selected.columns, X_scaled[0]))

In [ ]:
X_scaled 

array([[ 1.87307776,  1.14980335,  3.12617661, -1.42853637,  0.02945586,
        -0.71375385,  0.56631789, -0.56182371, -0.03941919,  0.08490949,
         0.45693323,  2.53959706,  0.21965619, -0.58974635,  0.52602006,
         0.16278859,  0.14007339,  0.36513335,  1.87609834,  0.35907305,
         0.44931753,  1.67004923,  0.1099692 ,  0.28425647,  0.02213914,
        -0.02350443,  1.28602763,  2.29296668, -2.15511131,  2.4388252 ,
         2.49697375,  0.09567412, -1.66955   ,  1.19522284,  2.29678392,
         2.14816421,  0.96958772,  1.63462377,  2.67353015,  0.49996433]])

In [ ]:
fighter_differences

{'year': 1.873077760255086,
 'fight_time_avg_diff': 1.1498033502884033,
 'days_since_debut_diff': 3.126176608339953,
 'win_pct_before_diff': -1.428536366016951,
 'sig_strikes_landed_per_minute_diff': 0.029455864217926172,
 'knockdown_avg_diff': -0.7137538532361234,
 'takedowns_avg_diff': 0.5663178857485662,
 'avg_ground_attempts_diff': -0.5618237083990046,
 'days_since_last_win_diff': -0.0394191908814744,
 'control_time_roll_5_diff': 0.08490948929002261,
 'avg_sub_attempts_diff': 0.45693323242913375,
 'losses_before_diff': 2.5395970589409473,
 'takedowns_ema_diff': 0.21965618969485498,
 'avg_ground_strikes_diff': -0.5897463454055173,
 'avg_takedowns_diff': 0.5260200567784586,
 'avg_clinch_attempts_diff': 0.16278858570393132,
 'avg_leg_attempts_diff': 0.14007339145986347,
 'takedowns_success_rate_roll_5_diff': 0.3651333543569672,
 'avg_body_strikes_diff': 1.8760983374162958,
 'avg_takedown_attempts_diff': 0.35907304838739945,
 'submission_avg_diff': 0.4493175271165769,
 'avg_body_attemp

In [ ]:
def load_and_predict_adaboost(fighter_differences, model_path='../models/sgdclassifier.pkl'):
    """
    Load AdaBoost model and make prediction using fighter differences
    
    Parameters:
    fighter_differences: dict with difference values (e.g., {'reach_diff': 2, 'height_diff': -1, ...})
    """
    # Load model
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    # Prepare features using only the differences
    features = list(fighter_differences.values())

    # Convert to numpy array
    X = np.array(features).reshape(1, -1)

    # Make prediction
    prediction = model.predict(X)[0]
    probabilities = model.predict_proba(X)[0]

    return {
        'prediction': prediction,
        'fighter_red_win_prob': probabilities[1] if len(probabilities) > 1 else probabilities[0],
        'fighter_blue_win_prob': probabilities[0] if len(probabilities) > 1 else 1 - probabilities[0],
        'confidence': max(probabilities),
        'model_name': 'AdaBoost'
    }

# Example usage with fighter differences
'''
fighter_differences = {
    'total_strikes_diff': 0.197245, 'sig_strikes_diff': -0.426000, 'head_strikes_diff': -0.688153,
    'ground_strikes_diff': -0.764567, 'ground_attempts_diff': -0.866484, 'sub_attempts_diff': 0.106177,
    'year': 1.392613, 'knockdowns_diff': 0.085592, 'control_time_diff': -0.100416,
    'total_strikes_attempts_diff': 0.067307, 'head_attempts_diff': -0.587013, 'distance_diff': -0.162938,
    'sig_attempts_diff': -0.392458, 'takedowns_diff': 0.123152, 'win_method_Submission': -0.495827,
    'fight_time_avg_diff': -0.310984, 'submission_avg_diff': 1.267790, 'days_since_debut_diff': 1.307316,
    'distance_attempts_diff': -0.074113, 'body_strikes_diff': 0.322581, 'takedowns_avg_diff': 0.734291,
    'sub_attempts_ema_diff': 0.285637, 'sig_attempts_ema_diff': 0.049346,
    'takedown_attempts_diff': 0.408287, 'time': 0.738943, 'knockdown_avg_diff': -0.067475,
    'body_attempts_diff': 0.340288, 'knockdowns_ema_diff': -0.368528, 'leg_attempts_diff': 0.125529,
    'clinch_strikes_diff': 0.207099
}'''

# Test the function
result = load_and_predict_adaboost(fighter_differences)
print("AdaBoost Prediction:")
print(f"Winner: Fighter {'Red' if result['prediction'] == 1 else 'Blue'}")
print(f"Fighter Red probability: {result['fighter_red_win_prob']:.3f}")
print(f"Fighter Blue probability: {result['fighter_blue_win_prob']:.3f}")
print(f"Confidence: {result['confidence']:.3f}")

C:\Users\yanbl\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SGDClassifier was fitted with feature names
  warnings.warn(


AttributeError: This 'SGDClassifier' has no attribute 'predict_proba'

In [18]:
def load_and_predict_model(fighter_differences, model_path='../models/sgdclassifier.pkl'):
    """
    Load model and make prediction using fighter differences

    Parameters:
    fighter_differences: dict with difference values (e.g., {'reach_diff': 2, 'height_diff': -1, ...})
    """
    # Load model
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    # Prepare features using only the differences
    features = list(fighter_differences.values())

    # Convert to numpy array
    X = np.array(features).reshape(1, -1)

    # Make prediction
    prediction = model.predict(X)[0]

    # Try predict_proba first, fallback to decision_function, then basic prediction
    try:
        probabilities = model.predict_proba(X)[0]
        fighter_red_prob = probabilities[1] if len(probabilities) > 1 else probabilities[0]
        fighter_blue_prob = probabilities[0] if len(probabilities) > 1 else 1 - probabilities[0]
        confidence = max(probabilities)
    except AttributeError:
        try:
            # Use decision_function for SGD
            decision_score = model.decision_function(X)[0]
            # Convert to pseudo-probability using sigmoid
            fighter_red_prob = 1 / (1 + np.exp(-decision_score))
            fighter_blue_prob = 1 - fighter_red_prob
            confidence = max(fighter_red_prob, fighter_blue_prob)
        except AttributeError:
            # Fallback: binary prediction only
            fighter_red_prob = float(prediction)
            fighter_blue_prob = 1.0 - float(prediction)
            confidence = 1.0

    return {
        'prediction': prediction,
        'fighter_red_win_prob': fighter_red_prob,
        'fighter_blue_win_prob': fighter_blue_prob,
        'confidence': confidence,
        'model_name': type(model).__name__
    }
'''
# Define your fighter_differences (example data from your comment)
fighter_differences = {
    'total_strikes_diff': 0.197245, 'sig_strikes_diff': -0.426000, 'head_strikes_diff': -0.688153,
    'ground_strikes_diff': -0.764567, 'ground_attempts_diff': -0.866484, 'sub_attempts_diff': 0.106177,
    'year': 1.392613, 'knockdowns_diff': 0.085592, 'control_time_diff': -0.100416,
    'total_strikes_attempts_diff': 0.067307, 'head_attempts_diff': -0.587013, 'distance_diff': -0.162938,
    'sig_attempts_diff': -0.392458, 'takedowns_diff': 0.123152, 'win_method_Submission': -0.495827,
    'fight_time_avg_diff': -0.310984, 'submission_avg_diff': 1.267790, 'days_since_debut_diff': 1.307316,
    'distance_attempts_diff': -0.074113, 'body_strikes_diff': 0.322581, 'takedowns_avg_diff': 0.734291,
    'sub_attempts_ema_diff': 0.285637, 'sig_attempts_ema_diff': 0.049346,
    'takedown_attempts_diff': 0.408287, 'time': 0.738943, 'knockdown_avg_diff': -0.067475,
    'body_attempts_diff': 0.340288, 'knockdowns_ema_diff': -0.368528, 'leg_attempts_diff': 0.125529,
    'clinch_strikes_diff': 0.207099
}
'''
# Test the function
result = load_and_predict_model(fighter_differences)
print("SGD Prediction:")
print(f"Winner: Fighter {'Red' if result['prediction'] == 1 else 'Blue'}")
print(f"Fighter Red probability: {result['fighter_red_win_prob']:.3f}")
print(f"Fighter Blue probability: {result['fighter_blue_win_prob']:.3f}")
print(f"Confidence: {result['confidence']:.3f}")

NameError: name 'fighter_differences' is not defined